# Libraries

In [1]:
import sys
import os
directory_path = os.path.abspath(os.path.join('..'))
utils_path = os.path.abspath(os.path.join('../utils'))
if directory_path not in sys.path:
    sys.path.append(directory_path)
    sys.path.append(utils_path)

In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from utils.Validator import *
from utils.Combinations import *
from utils.Network import *
pd.set_option('display.max_columns', None)
sns.set(rc={'figure.figsize':(11.7,8.27)})

# Data Consolidation

In [17]:
df_net_metrics = pd.read_csv(r"../data/interim/UserNetworksMetrics.csv")
df_net_metrics.head()

,UserId,UserName,DisplayName,PerformanceTier,Flg_Creacion,in_degree_followers,out_degree_followers,degree_cent_followers,pagerank_cent_followers,degree_teams,degree_cent_teams,eigenvector_cent_teams,degree_forums,degree_cent_forums,eigenvector_cent_forums
0,368,antgoldbloom,Anthony Goldbloom,2,5Y,6.0,6.0,0.003895,0.000003,4.0,0.000026,9.451808e-26,887.0,0.008758,0.001207
1,381,iguyon,Isabelle,2,5Y,0.0,0.0,0.000024,0.000003,NaN,NaN,NaN,113.0,0.001116,0.000052
2,389,anandjeyahar,Anand Jeyahar,1,5Y,0.0,0.0,0.000008,0.000003,1.0,0.000006,1.171515e-26,59.0,0.000583,0.000033
3,392,jmeynet,Julien Meynet,3,5Y,0.0,0.0,0.000004,0.000003,NaN,NaN,NaN,25.0,0.000247,0.000001
4,394,m4xl1n,m4xl1n,1,5Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df_user_att = pd.read_csv(r"../data/interim/UserAttributes.csv")
df_user_att.head()

,UserId,CTDCOMP,CTDCOMP_DS,CTDCOMP_Supervizado,CTDCOMP_NoSupervizado
0,1,1,1.0,0.0,1.0
1,62,1,0.0,0.0,0.0
2,368,101,63.0,44.0,62.0
3,381,64,2.0,1.0,2.0
4,387,1,0.0,0.0,0.0


In [19]:
df_user_achi = pd.read_csv(r"../data/interim/UserAchievements.csv")
df_user_achi.head()

,UserId,TotalGold_Comp,TotalSilver_Comp,TotalBronze_Comp,TotalGold_Scri,TotalSilver_Scri,TotalBronze_Scri,TotalGold_Disc,TotalSilver_Disc,TotalBronze_Disc,TotalGold_Data,TotalSilver_Data,TotalBronze_Data
0,368,0.0,0.0,0.0,0.0,3.0,13.0,14.0,13.0,139.0,1.0,3.0,6.0
1,381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,53.0,0.0,0.0,0.0
2,389,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,392,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
4,394,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
df_user_final = df_net_metrics.merge(df_user_att,how="left",on=["UserId"]).merge(df_user_achi,how="left",on=["UserId"])
df_user_final.head()

,UserId,UserName,DisplayName,PerformanceTier,Flg_Creacion,in_degree_followers,out_degree_followers,degree_cent_followers,pagerank_cent_followers,degree_teams,degree_cent_teams,eigenvector_cent_teams,degree_forums,degree_cent_forums,eigenvector_cent_forums,CTDCOMP,CTDCOMP_DS,CTDCOMP_Supervizado,CTDCOMP_NoSupervizado,TotalGold_Comp,TotalSilver_Comp,TotalBronze_Comp,TotalGold_Scri,TotalSilver_Scri,TotalBronze_Scri,TotalGold_Disc,TotalSilver_Disc,TotalBronze_Disc,TotalGold_Data,TotalSilver_Data,TotalBronze_Data
0,368,antgoldbloom,Anthony Goldbloom,2,5Y,6.0,6.0,0.003895,0.000003,4.0,0.000026,9.451808e-26,887.0,0.008758,0.001207,101.0,63.0,44.0,62.0,0.0,0.0,0.0,0.0,3.0,13.0,14.0,13.0,139.0,1.0,3.0,6.0
1,381,iguyon,Isabelle,2,5Y,0.0,0.0,0.000024,0.000003,NaN,NaN,NaN,113.0,0.001116,0.000052,64.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,53.0,0.0,0.0,0.0
2,389,anandjeyahar,Anand Jeyahar,1,5Y,0.0,0.0,0.000008,0.000003,1.0,0.000006,1.171515e-26,59.0,0.000583,0.000033,17.0,13.0,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,392,jmeynet,Julien Meynet,3,5Y,0.0,0.0,0.000004,0.000003,NaN,NaN,NaN,25.0,0.000247,0.000001,10.0,1.0,1.0,1.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
4,394,m4xl1n,m4xl1n,1,5Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
cols = df_user_final.columns[3:].tolist()
cols.remove('Flg_Creacion')

In [22]:
Validator.validar_numerico(df_user_final,cols)

,Variable,n,Missing,%Missing,Negativo,%Negativo,Unico,Media,Min,Max,IQRMAX,3STD,P50,P75,P90,P95,P99,Outlier IQR,%Outlier IQR,Outlier 3SD,%Outlier 3SD,Outlier P90,%Outlier P90,Outlier P95,%Outlier P95,Outlier P99,%Outlier P99
0,PerformanceTier,194147,0,0.00,0,0.0,4,1.08,1.0,4.00,1.0,2.08,1.0,1.0,1.0,2.0,3.00,13612,7.01,2445,1.26,13612,7.01,2445,1.26,349,0.18
0,in_degree_followers,194147,112354,57.87,0,0.0,248,5.13,0.0,2756.00,8.5,64.96,2.0,4.0,11.0,20.0,54.00,10991,5.66,607,0.31,7967,4.10,3856,1.99,813,0.42
0,out_degree_followers,194147,112354,57.87,0,0.0,248,5.13,0.0,2756.00,8.5,64.96,2.0,4.0,11.0,20.0,54.00,10991,5.66,607,0.31,7967,4.10,3856,1.99,813,0.42
0,degree_cent_followers,194147,112354,57.87,0,0.0,596,0.00,0.0,0.07,0.0,0.00,0.0,0.0,0.0,0.0,0.00,9732,5.01,234,0.12,7905,4.07,3985,2.05,815,0.42
0,pagerank_cent_followers,194147,112354,57.87,0,0.0,37294,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,12103,6.23,181,0.09,7718,3.98,4090,2.11,818,0.42
0,degree_teams,194147,155035,79.85,0,0.0,70,3.33,1.0,133.00,8.5,16.57,2.0,4.0,6.0,10.0,23.00,2392,1.23,701,0.36,3841,1.98,1634,0.84,361,0.19
0,degree_cent_teams,194147,155035,79.85,0,0.0,70,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,2392,1.23,701,0.36,3841,1.98,1634,0.84,361,0.19
0,eigenvector_cent_teams,194147,155035,79.85,0,0.0,13190,0.00,0.0,0.71,0.0,0.02,0.0,0.0,0.0,0.0,0.00,9671,4.98,7,0.00,3912,2.01,1956,1.01,392,0.20
0,degree_forums,194147,92863,47.83,0,0.0,1619,85.04,2.0,15622.00,200.0,733.41,30.0,86.0,189.0,282.0,920.17,8761,4.51,1498,0.77,10088,5.20,5061,2.61,1013,0.52
0,degree_cent_forums,194147,92863,47.83,0,0.0,1619,0.00,0.0,0.15,0.0,0.01,0.0,0.0,0.0,0.0,0.01,9006,4.64,1498,0.77,10088,5.20,5061,2.61,1013,0.52


In [23]:
df_user_final[df_user_final["TotalBronze_Disc"]==8420.00].head()

,UserId,UserName,DisplayName,PerformanceTier,Flg_Creacion,in_degree_followers,out_degree_followers,degree_cent_followers,pagerank_cent_followers,degree_teams,degree_cent_teams,eigenvector_cent_teams,degree_forums,degree_cent_forums,eigenvector_cent_forums,CTDCOMP,CTDCOMP_DS,CTDCOMP_Supervizado,CTDCOMP_NoSupervizado,TotalGold_Comp,TotalSilver_Comp,TotalBronze_Comp,TotalGold_Scri,TotalSilver_Scri,TotalBronze_Scri,TotalGold_Disc,TotalSilver_Disc,TotalBronze_Disc,TotalGold_Data,TotalSilver_Data,TotalBronze_Data
103257,3012786,mpwolke,Marília Prata,4,3Y_4Y,5.0,5.0,0.00889,0.000004,2.0,0.000013,8.020131e-09,15622.0,0.154241,0.38034,78.0,49.0,37.0,49.0,0.0,0.0,0.0,12.0,29.0,374.0,53.0,84.0,8420.0,1.0,10.0,93.0


In [24]:
df_user_final.to_csv(r"../data/processed//UserDataTotal.csv",index=False)